In [1]:
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel
# from openai import AsyncOpenAI
# import asyncio
# from typing import Optional
# from fastapi.responses import JSONResponse
# import os
# from dotenv import load_dotenv
####################################
from openai import AsyncOpenAI
import asyncio

In [2]:
async def call_assistant(client: AsyncOpenAI, prompt: str, assistant_id: str):
    
    # 建立 thread
    thread = await client.beta.threads.create()
    
    # 新增訊息
    await client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )
    
    # 執行 assistant
    run = await client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )
    
    # 等待完成
    while True:
        run = await client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        if run.status == "completed":
            break
        elif run.status == "failed":
            raise Exception("Assistant run failed")
        elif run.status == "expired":
            raise Exception("Assistant run expired")
        await asyncio.sleep(1)
    
    # 獲取回應
    messages = await client.beta.threads.messages.list(
        thread_id=thread.id
    )
    
    return messages.data[0].content[0].text.value


In [3]:
assistant_info_dict = {
    "rule_assistant_info":{
        "prompt":"你好，請幫我解答一個問題",
        "assistant_id":"asst_V42jgCpZV0NnumoEMadLr9b9"
    },
    "essay_assistant_info":{
        "prompt":"",
        "assistant_id":"asst_uXoFZL5iarQtwvS0GZvpLhoj"
    }
}

In [1]:
# 使用範例
async def main():
    try:
        client = AsyncOpenAI()
        rules_assistant_id = "asst_V42jgCpZV0NnumoEMadLr9b9"
        response = await call_assistant(
            "你好，請幫我解答一個問題",
            "asst_V42jgCpZV0NnumoEMadLr9b9"
        )
        print(response)
    except Exception as e:
        print(f"Error: {e}")

# # 執行
# if __name__ == "__main__":
#     asyncio.run(main())